|Variable|Variável     |Definição	                              |Chave                                         |
|--------|-------------|------------------------------------------|----------------------------------------------|
|survival|sobrevivência|Sobrevivência                             |0 = Não, 1 = Sim                              |
|pclass  |pclass	   |Classe de ingresso                        |1 = 1º, 2 = 2º, 3 = 3º                        |
|ex      |sexo         |Sexo                                      |                                              |
|Age     |Idade em anos| -                                        |                                              |
|sibsp   |sibsp        |Nº de irmãos / cônjuges a bordo do Titanic|	                                             |
|parch   |pergaminho   |Nº de pais / filhos a bordo do Titanic    |                                              |
|ticket  |bilhete      |Número do bilhete	                      |                                              |
|fare    |tarifa       |Tarifa de passageiro                      |	                                             |
|cabin   |cabine       |Número da cabine	                      |                                              |
|embarked|embarcou     |Porto de embarcação                       |C = Cherbourg, Q = Queenstown, S = Southampton|

In [30]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from collections import Counter

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [182]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

In [183]:
combine = [train, test]

In [184]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)

# pd.crosstab(train['Title'], train['Sex'])

In [185]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 
                                                'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()    

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [186]:
title = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title)
    dataset['Title'] = dataset['Title'].fillna(0)

In [187]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean()

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [188]:
for dataset in combine:
    idx = dataset['Age'].loc[dataset['Age']<1].index
    age = dataset['Age'].iloc[idx]*100
    dataset['Age'].iloc[idx] = age

    m = dataset['Age'].median()
    dataset['Age'] = dataset['Age'].fillna(m)

In [189]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 20, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 20) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
    dataset['Age'] = dataset['Age'].astype(int)

In [190]:
for dataset in combine:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

In [191]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

In [192]:
for dataset in combine:
    dataset.set_index('PassengerId', inplace=True)

In [193]:
train = train.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test = test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [194]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [176]:
x, y = train.drop('Survived', axis=1), train.Survived.copy()

In [177]:
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [178]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
dectree_clf = DecisionTreeClassifier(max_depth = 10, random_state = 0)
grad_clf = GradientBoostingClassifier()
navies_bayes = GaussianNB()

In [179]:
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf),
                                         ('svc', svm_clf), ('dt', dectree_clf),
                                         ('grad', grad_clf), ('navie', navies_bayes)], voting='hard')

In [180]:
voting_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',...
                                        

In [124]:
for clf in (log_clf,rnd_clf,svm_clf, dectree_clf,grad_clf, navies_bayes, voting_clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8116591928251121
RandomForestClassifier 0.8251121076233184
SVC 0.8609865470852018
DecisionTreeClassifier 0.8340807174887892
GradientBoostingClassifier 0.8565022421524664
GaussianNB 0.8340807174887892
VotingClassifier 0.8654708520179372


In [123]:
ptest = voting_clf.predict(test)
prev = pd.Series(ptest, index= test.index, name='Survived')
# prev.to_csv("modelo_ensemble_5.csv", header=True)    